In [1]:
import pandas as pd


crash_people = pd.read_csv('C:/Users/ekmho/Documents/GitHub/Europe Regional 2023 Datasets/Crashes/crash_info_people.csv')


C:\Users\ekmho\AppData\Local\Temp\ipykernel_9360\3002692852.py:4: DtypeWarning: Columns (3,9,10,11,18) have mixed types. Specify dtype option on import or set low_memory=False.
  crash_people = pd.read_csv('C:/Users/ekmho/Documents/GitHub/Europe Regional 2023 Datasets/Crashes/crash_info_people.csv')


In [2]:
crash_general = pd.read_csv('C:/Users/ekmho/Documents/GitHub/Europe Regional 2023 Datasets/Crashes/crash_info_general.csv')


C:\Users\ekmho\AppData\Local\Temp\ipykernel_9360\3111603682.py:1: DtypeWarning: Columns (97,98,99) have mixed types. Specify dtype option on import or set low_memory=False.
  crash_general = pd.read_csv('C:/Users/ekmho/Documents/GitHub/Europe Regional 2023 Datasets/Crashes/crash_info_general.csv')


In [3]:
crash_roadway =  pd.read_csv('C:/Users/ekmho/Documents/GitHub/Europe Regional 2023 Datasets/Crashes/crash_info_roadway.csv')


C:\Users\ekmho\AppData\Local\Temp\ipykernel_9360\2030635114.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  crash_roadway =  pd.read_csv('C:/Users/ekmho/Documents/GitHub/Europe Regional 2023 Datasets/Crashes/crash_info_roadway.csv')


In [4]:
crash_roadway

,CRN,RDWY_SEQ_NUM,ADJ_RDWY_SEQ,ROUTE,SEGMENT,OFFSET,RDWY_ORIENT,LANE_COUNT,SPEED_LIMIT,STREET_NAME,ROAD_OWNER,ACCESS_CTRL,RDWY_COUNTY
0,2019022023,4,3,NaN,NaN,NaN,W,2.0,25.0,HUNTING PARK AV,4,NaN,67
1,2019032454,3,3,NaN,NaN,NaN,N,2.0,25.0,N 5TH ST,4,NaN,67
2,2019034042,4,3,NaN,NaN,NaN,N,4.0,25.0,AMERICAN ST,4,NaN,67
3,2019035193,4,5,NaN,NaN,NaN,S,2.0,25.0,56TH ST,4,NaN,67
4,2019027003,5,5,NaN,NaN,NaN,E,2.0,20.0,FOX ST,4,NaN,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...
232147,2015002771,4,3,NaN,NaN,NaN,E,2.0,25.0,BELMAR ST,4,NaN,67
232148,2015003301,3,3,NaN,NaN,NaN,E,2.0,30.0,SPRUCE ST,4,NaN,67
232149,2015000348,3,3,1.0,30.0,0.0,N,2.0,35.0,CITY AV,2,3.0,67
232150,2015040562,3,3,2010.0,22.0,1862.0,W,2.0,25.0,CECIL B MOORE AV,2,3.0,67


Dropping irrelevant columns from all the dataframes

In [5]:
general_col_keep = ['CONS_ZONE_SPD_LIM', 'CRASH_MONTH', 'CRASH_YEAR', 'CRN', 'DEC_LAT', 'DEC_LONG', 'FATAL_COUNT', 
                    'ILLUMINATION', 'INTERSECT_TYPE', 'INTERSECT_RELATED', 'LANE_CLOSED', 'LN_CLOSE_DIR', 'LOCATION_TYPE'
                   'RDWY_SURF_TYPE_CD', 'ROAD_CONDITION', 'WEATHER1', 'WEATHER2']

crash_general = crash_general.drop(columns=[col for col in crash_general.columns if col not in general_col_keep])

In [6]:
roadway_col_keep = ['CRN', 'LANE_COUNT', 'SPEED_LIMIT']

crash_roadway = crash_roadway.drop(columns=[col for col in crash_roadway.columns if col not in roadway_col_keep])

In [7]:
person_col_keep = ['CRN', 'DVR_PED_CONDITION', 'PERSON_TYPE']

crash_people = crash_people.drop(columns=[col for col in crash_people.columns if col not in person_col_keep])

In [14]:
#Filtering for drivers that are in a normal condition
crash_people = crash_people[crash_people['PERSON_TYPE'] == 1]
crash_people = crash_people[crash_people['DVR_PED_CONDITION'] == 1]

In [16]:
non_human_caused_crashes = crash_people['CRN']

In [17]:
crash_general = crash_general[crash_general['CRN'].isin(non_human_caused_crashes)]
crash_general

,CRN,CONS_ZONE_SPD_LIM,CRASH_MONTH,CRASH_YEAR,DEC_LAT,DEC_LONG,FATAL_COUNT,ILLUMINATION,INTERSECT_TYPE,LANE_CLOSED,LN_CLOSE_DIR,ROAD_CONDITION,WEATHER1,WEATHER2
43,2010008992,NaN,1,2010,39.9600,-75.1800,0,3.0,0,2,3.0,1,3,NaN
60,2010010428,NaN,1,2010,39.9325,-75.2000,0,3.0,0,1,4.0,1,3,NaN
70,2010011506,NaN,1,2010,40.0732,-75.1894,0,3.0,0,1,1.0,1,3,NaN
101,2010011581,NaN,1,2010,39.9080,-75.1732,0,3.0,1,0,NaN,9,7,NaN
116,2010011618,NaN,1,2010,39.9538,-75.2408,0,3.0,0,0,NaN,9,7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132889,2022005014,NaN,12,2021,39.9438,-75.1427,0,3.0,0,0,NaN,1,3,3.0
132903,2022006555,NaN,11,2021,39.9699,-75.1255,0,3.0,7,2,1.0,1,3,NaN
132920,2022010813,NaN,12,2021,40.0101,-75.1362,1,1.0,1,2,1.0,1,4,4.0
132963,2022014503,NaN,7,2021,39.9489,-75.2171,0,3.0,0,2,4.0,1,3,3.0


In [20]:
pip install folium


     -------------------------------------- 102.3/102.3 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [26]:
#Looking at the NAN counts for each column
nan_counts = crash_general.isna().sum()
print(nan_counts)

CRN                     0
CONS_ZONE_SPD_LIM    5319
CRASH_MONTH             0
CRASH_YEAR              0
DEC_LAT                24
DEC_LONG               24
FATAL_COUNT             0
ILLUMINATION            0
INTERSECT_TYPE          0
LANE_CLOSED             0
LN_CLOSE_DIR         3183
ROAD_CONDITION          0
WEATHER1                0
WEATHER2             5017
dtype: int64


In [28]:
]Removing the 
crash_general.dropna(subset=['DEC_LAT', 'DEC_LONG'], inplace=True)


,CRN,CONS_ZONE_SPD_LIM,CRASH_MONTH,CRASH_YEAR,DEC_LAT,DEC_LONG,FATAL_COUNT,ILLUMINATION,INTERSECT_TYPE,LANE_CLOSED,LN_CLOSE_DIR,ROAD_CONDITION,WEATHER1,WEATHER2


Visualising the distribution of accidents not caused by driver condition

In [22]:
import folium
from folium.plugins import HeatMap

# create a map centered on Philadelphia
map_center = [39.9526, -75.1652]  # coordinates of Philadelphia
crash_map = folium.Map(location=map_center, zoom_start=10)

# create a list of coordinates from the crash data
crash_locations = crash_general[['DEC_LAT', 'DEC_LONG']].values.tolist()

# create a heatmap layer from the coordinates
heatmap_layer = HeatMap(crash_locations)

# add the heatmap layer to the map
heatmap_layer.add_to(crash_map)

# display the map
crash_map

ValueError: Location values cannot contain NaNs.